<a href="https://colab.research.google.com/github/DanielCoser/Webscrapping-ML/blob/main/Manipula%C3%A7%C3%A3o_de_Dados_Calculadora_de_Aluguel_Coleta_de_Dados_Daniel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [147]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
from time import sleep

zonas = ['sul', 'oeste', 'leste', 'norte']
paginas = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193']
url_ml = 'https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/'

def coletando_dados(url, zona):

  resposta = requests.get(url).content
  sopa = BeautifulSoup(resposta)

  result_set_precos = sopa.find_all('span', {'class': 'price-tag-fraction'})
  result_set_area_quartos = sopa.find_all('ul', class_='ui-search-card-attributes ui-search-item__group__element')
  result_set_enderecos = sopa.find_all('span', class_='ui-search-item__group__element ui-search-item__location')

  lista_precos = []
  lista_area_quartos = []
  lista_enderecos = []
  lista_zonas = []

  for preco, area_quarto, endereco in zip(result_set_precos, result_set_area_quartos, result_set_enderecos):
    lista_precos.append(preco.text)
    lista_area_quartos.append(area_quarto.text)
    lista_enderecos.append(endereco.text)
    lista_zonas.append(zona)
    
  dados = {'zonas': lista_zonas,
           'areas_quartos': lista_area_quartos,
           'enderecos': lista_enderecos,
           'precos': lista_precos}

  df = pd.DataFrame(dados)
  return df

In [148]:
lista_dados = []

for zona in zonas:
  for pagina in paginas:
    url = f'https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-{zona}/{pagina}'
    print('Coletando dados de', url)
    df = coletando_dados(url, zona)
    lista_dados.append(df)
    sleep(2) # lembrar de usar sleep para não sobrecarregar o site de acessos
print('Dados coletados!')

Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_49
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_97
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_145
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_193
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-oeste/
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-oeste/_Desde_49
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-oeste/_Desde_97
Coletando dados de https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-

In [149]:
df_dados = pd.concat(lista_dados)
df_dados
df_dados['areas_quartos'] = df_dados['areas_quartos'].str.replace(',','.')

In [150]:
df_dados.to_csv('dados_mercado_livre.csv', index=False) # index=False para tirar o unnamed - coluna adicional de indices

In [151]:
import re
df_ml = pd.read_csv('dados_mercado_livre.csv', thousands='.')
df_ml.head()

,zonas,areas_quartos,enderecos,precos
0,sul,45 m² construídos1 quarto,"Rua Doutor Fadlo Haidar,\n 75, ...",3200
1,sul,88 m² construídos3 quartos,"R Apotribu, Parque Imperial, São Paulo Zona Su...",2800
2,sul,83 m² construídos3 quartos,"Av Mascote, Vila Mascote, São Paulo Zona Sul, ...",3500
3,sul,51 m² construídos2 quartos,"Rua Taquacetuba, Vila Parque Jabaquara, São Pa...",1200
4,sul,48 m² construídos1 quarto,"Rua Charles Spencer Chaplin, Vila Andrade, São...",1906


In [152]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960 entries, 0 to 959
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   zonas          960 non-null    object
 1   areas_quartos  960 non-null    object
 2   enderecos      960 non-null    object
 3   precos         960 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 30.1+ KB


In [153]:
linha_0 = df_ml['areas_quartos'][0]
df_ml['areas_quartos'][706]

'135.000 m² construídos3 quartos'

In [154]:
def extrair_area(entrada):
  if 'm²' in entrada:
    # uma solução é com o find que devolve o indíce e aí seleciona um pedaço
    # outra solução é dividir o texto onde tem m²
    area = entrada.split(' m²')[0]
  else:
    area = np.nan
  return area

In [155]:
linha_0_va = '2 quartos'
extrair_area(linha_0_va)

nan

In [156]:
df_ml['areas'] =  df_ml['areas_quartos'].apply(extrair_area).astype(float)
df_ml['precos'] = df_ml['precos'].astype(float)

In [157]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960 entries, 0 to 959
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   zonas          960 non-null    object 
 1   areas_quartos  960 non-null    object 
 2   enderecos      960 non-null    object 
 3   precos         960 non-null    float64
 4   areas          960 non-null    float64
dtypes: float64(2), object(3)
memory usage: 37.6+ KB


In [158]:
texto = linha_0
padrao = '(\d+) quarto' # regex que significa um grupo de um ou mais dígitos
padrao1 = '(\d+)'

In [159]:
re.findall(padrao1, texto) # exemplo - aqui já podemos checar e dividir direto em areas e quartos

['45', '1']

In [162]:
def extrair_quarto(entrada):

  padrao = '(\d+) quarto'
  
  if 'quarto' in entrada:
    n_quartos = re.findall(padrao, entrada)[0]
  else:
    n_quartos = np.nan
  
  return n_quartos

In [164]:
df_ml['quartos'] = df_ml['areas_quartos'].apply(extrair_quarto).astype(float)

In [165]:
df_ml.head()

,zonas,areas_quartos,enderecos,precos,areas,quartos
0,sul,45 m² construídos1 quarto,"Rua Doutor Fadlo Haidar,\n 75, ...",3200.0,45.0,1.0
1,sul,88 m² construídos3 quartos,"R Apotribu, Parque Imperial, São Paulo Zona Su...",2800.0,88.0,3.0
2,sul,83 m² construídos3 quartos,"Av Mascote, Vila Mascote, São Paulo Zona Sul, ...",3500.0,83.0,3.0
3,sul,51 m² construídos2 quartos,"Rua Taquacetuba, Vila Parque Jabaquara, São Pa...",1200.0,51.0,2.0
4,sul,48 m² construídos1 quarto,"Rua Charles Spencer Chaplin, Vila Andrade, São...",1906.0,48.0,1.0


In [166]:
df_ml = df_ml.drop('areas_quartos', axis=1)

In [167]:
df_ml.head()

,zonas,enderecos,precos,areas,quartos
0,sul,"Rua Doutor Fadlo Haidar,\n 75, ...",3200.0,45.0,1.0
1,sul,"R Apotribu, Parque Imperial, São Paulo Zona Su...",2800.0,88.0,3.0
2,sul,"Av Mascote, Vila Mascote, São Paulo Zona Sul, ...",3500.0,83.0,3.0
3,sul,"Rua Taquacetuba, Vila Parque Jabaquara, São Pa...",1200.0,51.0,2.0
4,sul,"Rua Charles Spencer Chaplin, Vila Andrade, São...",1906.0,48.0,1.0


In [168]:
df_ml.to_csv('dados_tratados_mercado_livre.csv', index=False)